In [1]:
#use GPU
import torch
torch.cuda.is_available()

True

Set flair

In [2]:
import flair

In [3]:
# for test
#https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md

from flair.data import Sentence
from flair.models import SequenceTagger

# load the NER tagger
tagger_de = SequenceTagger.load('de-ner-large')

text_de = "Erledigung des Prüfungsberichtes Der Stadtrechnungshof Wien unterzog die Gebarung der Vienna Film Commission GmbH in den Jahren 2015 bis 2017 einer Prüfung. Der diesbezügliche Bericht des Stadtrechnungshofes Wien wurde am 8. Oktober 2019 veröffentlicht und im Rahmen der Sitzung des Stadtrechnungshofausschusses vom 16. Oktober 2019, Ausschusszahl 63/19 mit Beschluss zur Kenntnis genommen."
# make a sentence
sentence_de = Sentence(text_de)

# run NER over sentence
tagger_de.predict(sentence_de)

# print the sentence with all annotations
print(sentence_de)

print('The following NER tags are found:')

# iterate over entities and print each
for entity in sentence_de.get_spans('ner'):
    print(entity)

/root/.local/lib/python3.7/site-packages/huggingface_hub/file_download.py:591: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-10-11 13:08:46,020 loading file /root/.flair/models/ner-german-large/6b8de9edd73722050be2547acf64c037b2df833c6e8f0e88934de08385e26c1e.4b0797effcc6ebb1889d5d29784b97f0a099c1569b319d87d7c387e44e2bba48


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2022-10-11 13:09:20,783 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>
Sentence: "Erledigung des Prüfungsberichtes Der Stadtrechnungshof Wien unterzog die Gebarung der Vienna Film Commission GmbH in den Jahren 2015 bis 2017 einer Prüfung . Der diesbezügliche Bericht des Stadtrechnungshofes Wien wurde am 8. Oktober 2019 veröffentlicht und im Rahmen der Sitzung des Stadtrechnungshofausschusses vom 16. Oktober 2019 , Ausschusszahl 63 / 19 mit Beschluss zur Kenntnis genommen ." → ["Wien"/LOC, "Vienna Film Commission GmbH"/ORG, "Wien"/LOC]
The following NER tags are found:
Span[5:6]: "Wien" → LOC (1.0)
Span[10:14]: "Vienna Film Commission GmbH" → ORG (1.0)
Span[28:29]: "Wien" → LOC (1.0)


Reading Your Own Sequence Labeling Dataset

In [4]:
# 1. get the corpus
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'pos', 2: 'ner'}

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus("", columns,
                              train_file='train_flair.txt',
                              test_file='test_flair.txt')
#                               dev_file = 'dev_flair.txt')

# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-10-11 13:10:29,449 Reading data from .
2022-10-11 13:10:29,461 Train: train_flair.txt
2022-10-11 13:10:29,463 Dev: None
2022-10-11 13:10:29,471 Test: test_flair.txt
2022-10-11 13:10:34,854 Computing label dictionary. Progress:


2334it [00:00, 46239.73it/s]

2022-10-11 13:10:34,916 Dictionary created for label 'ner' with 2 values: ORG (seen 2066 times)
Dictionary with 2 tags: <unk>, ORG


In [7]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# 4. initialize embedding stack with Flair and GloVe
embedding_types = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.1,
              mini_batch_size=16,
              max_epochs=10)

2022-10-11 13:18:27,433 SequenceTagger predicts: Dictionary with 5 tags: O, S-ORG, B-ORG, E-ORG, I-ORG
2022-10-11 13:18:27,694 ----------------------------------------------------------------------------------------------------
2022-10-11 13:18:27,697 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
 

/root/.local/lib/python3.7/site-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there "


2022-10-11 13:18:30,683 epoch 1 - iter 14/146 - loss 0.56249803 - samples/sec: 76.61 - lr: 0.100000
2022-10-11 13:18:33,328 epoch 1 - iter 28/146 - loss 0.37171176 - samples/sec: 84.80 - lr: 0.100000
2022-10-11 13:18:36,135 epoch 1 - iter 42/146 - loss 0.29607390 - samples/sec: 79.88 - lr: 0.100000
2022-10-11 13:18:39,035 epoch 1 - iter 56/146 - loss 0.24566340 - samples/sec: 77.38 - lr: 0.100000
2022-10-11 13:18:41,737 epoch 1 - iter 70/146 - loss 0.22201969 - samples/sec: 83.02 - lr: 0.100000
2022-10-11 13:18:44,792 epoch 1 - iter 84/146 - loss 0.20042550 - samples/sec: 73.39 - lr: 0.100000
2022-10-11 13:18:47,568 epoch 1 - iter 98/146 - loss 0.18506481 - samples/sec: 80.78 - lr: 0.100000
2022-10-11 13:18:50,366 epoch 1 - iter 112/146 - loss 0.17451829 - samples/sec: 80.17 - lr: 0.100000
2022-10-11 13:18:52,939 epoch 1 - iter 126/146 - loss 0.16286257 - samples/sec: 87.16 - lr: 0.100000
2022-10-11 13:18:55,467 epoch 1 - iter 140/146 - loss 0.15489250 - samples/sec: 88.70 - lr: 0.1000

100%|██████████| 17/17 [00:03<00:00,  4.98it/s]

2022-10-11 13:19:00,106 Evaluating as a multi-label problem: False
2022-10-11 13:19:00,120 DEV : loss 0.055510684847831726 - f1-score (micro avg)  0.5814
2022-10-11 13:19:00,187 BAD EPOCHS (no improvement): 0
2022-10-11 13:19:00,189 saving best model


...

2022-10-11 13:24:08,753 Evaluating as a multi-label problem: False
2022-10-11 13:24:08,768 DEV : loss 0.022004157304763794 - f1-score (micro avg)  0.7433
2022-10-11 13:24:08,831 BAD EPOCHS (no improvement): 1


2022-10-11 13:24:10,681 ----------------------------------------------------------------------------------------------------
2022-10-11 13:24:10,684 loading file resources/taggers/sota-ner-flair/best-model.pt
2022-10-11 13:24:11,715 SequenceTagger predicts: Dictionary with 7 tags: O, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


100%|██████████| 70/70 [01:25<00:00,  1.22s/it]

2022-10-11 13:25:37,140 Evaluating as a multi-label problem: False
2022-10-11 13:25:37,159 0.7491	0.6941	0.7205	0.5632
2022-10-11 13:25:37,160 
Results:
- F-score (micro) 0.7205
- F-score (macro) 0.7205
- Accuracy 0.5632

By class:
              precision    recall  f1-score   support

         ORG     0.7491    0.6941    0.7205       925

   micro avg     0.7491    0.6941    0.7205       925
   macro avg     0.7491    0.6941    0.7205       925
weighted avg     0.7491    0.6941    0.7205       925

2022-10-11 13:25:37,163 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7205387205387206,
 'dev_score_history': [0.5813953488372093,
  0.6226415094339622,
  0.6231884057971014,
  0.6136363636363636,
  0.6494117647058824,
  0.6480186480186481,
  0.6651053864168619,
  0.7064439140811457,
  0.7594339622641508,
  0.7432762836185819],
 'train_loss_history': [0.1527855202287295,
  0.06633509599347677,
  0.05386010580421053,
  0.04868079927052878,
  0.045621785198081195,
  0.04234102438319594,
  0.041796973833808364,
  0.03858111388641701,
  0.03751684271699241,
  0.03578229459732843],
 'dev_loss_history': [0.055510684847831726,
  0.03968806192278862,
  0.035438667982816696,
  0.03667806461453438,
  0.02828693948686123,
  0.02700667642056942,
  0.028297198936343193,
  0.02517099492251873,
  0.023691877722740173,
  0.022004157304763794]}